# Capstone Project Working Notebook: David

In [1]:
import pandas as pd
import numpy as np

import csv


In [2]:
ipums_earnings = pd.read_csv('~/codeup/external_data_tap/data_capstone_project/ipums_earnings.csv')

In [3]:
ipums_earnings.shape

(3480188, 37)

In [4]:
def get_mass_majors_df():
    '''Function to initially pull and merge the two (2) needed
    College Scorecard tables for period 2018-2019.'''
    filename_01 = "~/codeup/external_data_tap/data_capstone_project/FieldOfStudyData1718_1819_PP.csv"
    filename_02 = "~/codeup/external_data_tap/data_capstone_project/MERGED2018_19_PP.csv"
    df_parent = pd.read_csv(filename_01)
    df_child = pd.read_csv(filename_02, low_memory=False)
    df_parent["UNITID"] = df_parent["UNITID"].astype("Int64", errors='ignore')
    df_child["UNITID"] = df_child["UNITID"].astype("Int64", errors='ignore')
    df = df_parent.merge(
    df_child,
    how = "left",
    on = "UNITID",
    copy = False
    )
    print(f'dataframe shape: {df.shape}')
    return df


In [5]:
df = get_mass_majors_df()

dataframe shape: (224849, 3109)


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 224849 entries, 0 to 224848
Columns: 3109 entries, UNITID to MD_EARN_WNE_MALE1_P10
dtypes: Int64(1), float64(1687), int64(4), object(1417)
memory usage: 5.2+ GB


In [7]:
df.head()

,UNITID,OPEID6_x,INSTNM_x,CONTROL_x,MAIN_x,CIPCODE,CIPDESC,CREDLEV,CREDDESC,IPEDSCOUNT1,...,COUNT_WNE_MALE1_P8,MD_EARN_WNE_MALE1_P8,GT_THRESHOLD_P10,MD_EARN_WNE_INC1_P10,MD_EARN_WNE_INC2_P10,MD_EARN_WNE_INC3_P10,MD_EARN_WNE_INDEP1_P10,MD_EARN_WNE_INDEP0_P10,MD_EARN_WNE_MALE0_P10,MD_EARN_WNE_MALE1_P10
0,100654,1002,Alabama A & M University,Public,1,100,"Agriculture, General.",3,Bachelors Degree,NaN,...,834.0,36639.0,0.6044,34076.0,35597.0,43145.0,40299.0,35424.0,36050.0,36377.0
1,100654,1002,Alabama A & M University,Public,1,109,Animal Sciences.,3,Bachelors Degree,5.0,...,834.0,36639.0,0.6044,34076.0,35597.0,43145.0,40299.0,35424.0,36050.0,36377.0
2,100654,1002,Alabama A & M University,Public,1,110,Food Science and Technology.,3,Bachelors Degree,9.0,...,834.0,36639.0,0.6044,34076.0,35597.0,43145.0,40299.0,35424.0,36050.0,36377.0
3,100654,1002,Alabama A & M University,Public,1,110,Food Science and Technology.,5,Master's Degree,5.0,...,834.0,36639.0,0.6044,34076.0,35597.0,43145.0,40299.0,35424.0,36050.0,36377.0
4,100654,1002,Alabama A & M University,Public,1,110,Food Science and Technology.,6,Doctoral Degree,1.0,...,834.0,36639.0,0.6044,34076.0,35597.0,43145.0,40299.0,35424.0,36050.0,36377.0


In [8]:
df.EARN_NE_MDN_3YR.value_counts()

PrivacySuppressed    176471
0                        70
38724                    36
35001                    34
33866                    34
                      ...  
30330                     1
59089                     1
74951                     1
83710                     1
64308                     1
Name: EARN_NE_MDN_3YR, Length: 29997, dtype: int64

In [9]:
df.NPT4_PUB.value_counts()

30996.0    521
22682.0    453
17479.0    442
28758.0    428
11644.0    415
          ... 
29652.0      1
16054.0      1
10604.0      1
5533.0       1
15476.0      1
Name: NPT4_PUB, Length: 1784, dtype: int64

In [10]:
df.ADMCON7.isnull().sum()

98312

In [11]:
df.head()

,UNITID,OPEID6_x,INSTNM_x,CONTROL_x,MAIN_x,CIPCODE,CIPDESC,CREDLEV,CREDDESC,IPEDSCOUNT1,...,COUNT_WNE_MALE1_P8,MD_EARN_WNE_MALE1_P8,GT_THRESHOLD_P10,MD_EARN_WNE_INC1_P10,MD_EARN_WNE_INC2_P10,MD_EARN_WNE_INC3_P10,MD_EARN_WNE_INDEP1_P10,MD_EARN_WNE_INDEP0_P10,MD_EARN_WNE_MALE0_P10,MD_EARN_WNE_MALE1_P10
0,100654,1002,Alabama A & M University,Public,1,100,"Agriculture, General.",3,Bachelors Degree,NaN,...,834.0,36639.0,0.6044,34076.0,35597.0,43145.0,40299.0,35424.0,36050.0,36377.0
1,100654,1002,Alabama A & M University,Public,1,109,Animal Sciences.,3,Bachelors Degree,5.0,...,834.0,36639.0,0.6044,34076.0,35597.0,43145.0,40299.0,35424.0,36050.0,36377.0
2,100654,1002,Alabama A & M University,Public,1,110,Food Science and Technology.,3,Bachelors Degree,9.0,...,834.0,36639.0,0.6044,34076.0,35597.0,43145.0,40299.0,35424.0,36050.0,36377.0
3,100654,1002,Alabama A & M University,Public,1,110,Food Science and Technology.,5,Master's Degree,5.0,...,834.0,36639.0,0.6044,34076.0,35597.0,43145.0,40299.0,35424.0,36050.0,36377.0
4,100654,1002,Alabama A & M University,Public,1,110,Food Science and Technology.,6,Doctoral Degree,1.0,...,834.0,36639.0,0.6044,34076.0,35597.0,43145.0,40299.0,35424.0,36050.0,36377.0


In [12]:
roi_df = df[[
    'UNITID',
    'INSTNM_x',
    'CITY',
    'EARN_MDN_HI_1YR',
    'EARN_MDN_HI_2YR',
    'NPT4_PUB',
    'NPT4_PRIV',
    'NPT4_PROG',
    'NPT4_OTHER',
    'COSTT4_A',
    'COSTT4_P',
    'TUITIONFEE_IN',
    'TUITIONFEE_OUT',
    'PCTPELL',
    'PCTFLOAN',
    'BOOKSUPPLY',
    'ROOMBOARD_ON',
    'OTHEREXPENSE_ON',
    'ROOMBOARD_OFF',
    'OTHEREXPENSE_OFF',
    'OTHEREXPENSE_FAM',
]]

In [13]:
general_df = df[[
    'UNITID',
    'INSTNM_x',
    'CITY',
    'STABBR',
    'ZIP',
    'REGION',
    'CIPCODE',
    'CIPDESC',
    'CREDLEV',
    'CREDDESC',
    'ACCREDAGENCY',
    'NUMBRANCH',
    'SCH_DEG',
    'PREDDEG',
    'CCBASIC',
    'CCUGPROF',
    'CCSIZSET',
    'HBCU',
    'DISTANCEONLY',
    'OPEFLAG',
    'ADM_RATE', 
    'ADMCON7',
    'SAT_AVG', 
    'ACTCMMID',
    'UGDS_WHITE',
    'UGDS_BLACK',
    'UGDS_HISP',
    'UGDS_ASIAN',
    'UGDS_AIAN',
    'UGDS_NHPI',
    'UGDS_2MOR',
    'UGDS_NRA',
    'UGDS_UNKN',
    'UGDS_WHITENH',
    'UGDS_BLACKNH',
    'UGDS_API',
    'UGDS_AIANOLD',
    'C150_4_WHITE',
    'C150_4_BLACK',
    'C150_4_HISP',
    'C150_4_ASIAN',
    'C150_4_AIAN',
    'C150_4_NHPI',
    'C150_4_2MOR',
    'C150_4_NRA',
    'C150_4_UNKN',
    'C150_4_WHITENH',
    'C150_4_BLACKNH',
    'C150_4_API',
    'C150_4_AIANOLD',
    'C150_4_HISPOLD',
    'PCIP01',
    'PCIP03',
    'PCIP04',
    'PCIP05',
    'PCIP09',
    'PCIP10',
    'PCIP11',
    'PCIP12',
    'PCIP13',
    'PCIP14',
    'PCIP15',
    'PCIP16',
    'PCIP19',
    'PCIP22',
    'PCIP23',
    'PCIP24',
    'PCIP25',
    'PCIP26',
    'PCIP27',
    'PCIP29',
    'PCIP30',
    'PCIP31',
    'PCIP38',
    'PCIP39',
    'PCIP40',
    'PCIP41',
    'PCIP42',
    'PCIP43',
    'PCIP44',
    'PCIP45',
    'PCIP46',
    'PCIP47',
    'PCIP48',
    'PCIP49',
    'PCIP50',
    'PCIP51',
    'PCIP52',
    'PCIP54',
    'PPTUG_EF',
    'NUM4_PUB',
    'NUM4_PRIV',
    'NUM4_PROG',
    'NUM4_OTHER',
    'AVGFACSAL',
    'C150_4',
    'C150_L4',
    'PFTFTUG1_EF',
    'RET_FT4',
    'RET_FTL4',
    'RET_PT4',
    'RET_PTL4',
    'PAR_ED_PCT_1STGEN',
    'FEMALE',
    'MARRIED',
    'DEPENDENT',
    'VETERAN',
    'FIRST_GEN',
    'FAMINC',
    'MD_FAMINC',
    'D_PCTPELL_PCTFLOAN',
    'UGNONDS',
    'GRADS',
    'PRGMOFR'
]]

In [14]:
loan_df = df[[
    'COUNT_ED',
    'LOAN_EVER',
    'PELL_EVER',
    'FTFTPCTPELL',
    'FTFTPCTFLOAN',
    'RPY_1YR_RT',
    'COMPL_RPY_1YR_RT',
    'NONCOM_RPY_1YR_RT',
    'LO_INC_RPY_1YR_RT',
    'MD_INC_RPY_1YR_RT',
    'HI_INC_RPY_1YR_RT',
    'DEP_RPY_1YR_RT',
    'IND_RPY_1YR_RT',
    'PELL_RPY_1YR_RT',
    'NOPELL_RPY_1YR_RT',
    'FEMALE_RPY_1YR_RT',
    'MALE_RPY_1YR_RT',
    'FIRSTGEN_RPY_1YR_RT',
    'NOTFIRSTGEN_RPY_1YR_RT',
    'RPY_3YR_RT',
    'COMPL_RPY_3YR_RT',
    'NONCOM_RPY_3YR_RT',
    'LO_INC_RPY_3YR_RT',
    'MD_INC_RPY_3YR_RT',
    'HI_INC_RPY_3YR_RT',
    'DEP_RPY_3YR_RT',
    'IND_RPY_3YR_RT',
    'PELL_RPY_3YR_RT',
    'NOPELL_RPY_3YR_RT',
    'FEMALE_RPY_3YR_RT',
    'MALE_RPY_3YR_RT',
    'FIRSTGEN_RPY_3YR_RT',
    'NOTFIRSTGEN_RPY_3YR_RT',
    'RPY_5YR_RT',
    'COMPL_RPY_5YR_RT',
    'NONCOM_RPY_5YR_RT',
    'LO_INC_RPY_5YR_RT',
    'MD_INC_RPY_5YR_RT',
    'HI_INC_RPY_5YR_RT',
    'DEP_RPY_5YR_RT',
    'IND_RPY_5YR_RT',
    'PELL_RPY_5YR_RT',
    'NOPELL_RPY_5YR_RT',
    'FEMALE_RPY_5YR_RT',
    'MALE_RPY_5YR_RT',
    'FIRSTGEN_RPY_5YR_RT',
    'NOTFIRSTGEN_RPY_5YR_RT',
    'RPY_7YR_RT',
    'COMPL_RPY_7YR_RT',
    'NONCOM_RPY_7YR_RT',
    'LO_INC_RPY_7YR_RT',
    'MD_INC_RPY_7YR_RT',
    'HI_INC_RPY_7YR_RT',
    'DEP_RPY_7YR_RT',
    'IND_RPY_7YR_RT',
    'PELL_RPY_7YR_RT',
    'NOPELL_RPY_7YR_RT',
    'FEMALE_RPY_7YR_RT',
    'MALE_RPY_7YR_RT',
    'FIRSTGEN_RPY_7YR_RT',
    'NOTFIRSTGEN_RPY_7YR_RT',
    'DEBT_MDN',
    'GRAD_DEBT_MDN',
    'WDRAW_DEBT_MDN',
    'LO_INC_DEBT_MDN',
    'MD_INC_DEBT_MDN',
    'HI_INC_DEBT_MDN',
    'DEP_DEBT_MDN',
    'IND_DEBT_MDN',
    'PELL_DEBT_MDN',
    'NOPELL_DEBT_MDN',
    'FEMALE_DEBT_MDN',
    'MALE_DEBT_MDN',
    'FIRSTGEN_DEBT_MDN',
    'NOTFIRSTGEN_DEBT_MDN',
    'CUML_DEBT_N',
    'CUML_DEBT_P90',
    'CUML_DEBT_P75',
    'CUML_DEBT_P25',
    'CUML_DEBT_P10',
    'DEBT_ALL_PP_EVAL_MEAN',
    'DEBT_ALL_PP_EVAL_MDN',
    'DEBT_PELL_PP_EVAL_MEAN',
    'DEBT_PELL_PP_EVAL_MDN',
    'DEBT_ALL_STGP_EVAL_MEAN',
    'DEBT_ALL_STGP_EVAL_MDN', 
    'DEBT_PELL_STGP_EVAL_MEAN',
    'DEBT_PELL_STGP_EVAL_MDN',
    'DEBT_NOPELL_STGP_EVAL_MEAN',
    'DEBT_NOPELL_STGP_EVAL_MDN',
    'DEBT_ALL_PP_EVAL_MDN10YRPAY',
    'DEBT_ALL_STGP_EVAL_MDN10YRPAY'
]]

In [15]:
roi_df.shape

(224849, 21)

In [16]:
roi_df.head()

,UNITID,INSTNM_x,CITY,EARN_MDN_HI_1YR,EARN_MDN_HI_2YR,NPT4_PUB,NPT4_PRIV,NPT4_PROG,NPT4_OTHER,COSTT4_A,...,TUITIONFEE_IN,TUITIONFEE_OUT,PCTPELL,PCTFLOAN,BOOKSUPPLY,ROOMBOARD_ON,OTHEREXPENSE_ON,ROOMBOARD_OFF,OTHEREXPENSE_OFF,OTHEREXPENSE_FAM
0,100654,Alabama A & M University,Normal,PrivacySuppressed,NaN,14444.0,NaN,NaN,NaN,22489.0,...,9744.0,18354.0,0.7067,0.7503,1600.0,9128.0,2990.0,9128.0,2990.0,2990.0
1,100654,Alabama A & M University,Normal,PrivacySuppressed,NaN,14444.0,NaN,NaN,NaN,22489.0,...,9744.0,18354.0,0.7067,0.7503,1600.0,9128.0,2990.0,9128.0,2990.0,2990.0
2,100654,Alabama A & M University,Normal,PrivacySuppressed,NaN,14444.0,NaN,NaN,NaN,22489.0,...,9744.0,18354.0,0.7067,0.7503,1600.0,9128.0,2990.0,9128.0,2990.0,2990.0
3,100654,Alabama A & M University,Normal,PrivacySuppressed,NaN,14444.0,NaN,NaN,NaN,22489.0,...,9744.0,18354.0,0.7067,0.7503,1600.0,9128.0,2990.0,9128.0,2990.0,2990.0
4,100654,Alabama A & M University,Normal,PrivacySuppressed,NaN,14444.0,NaN,NaN,NaN,22489.0,...,9744.0,18354.0,0.7067,0.7503,1600.0,9128.0,2990.0,9128.0,2990.0,2990.0


In [17]:
roi_df.EARN_MDN_HI_1YR.value_counts()

PrivacySuppressed    170325
36958                    37
25606                    35
28574                    35
42593                    35
                      ...  
61481                     1
117773                    1
7704                      1
54765                     1
16994                     1
Name: EARN_MDN_HI_1YR, Length: 32073, dtype: int64

In [18]:
roi_df.EARN_MDN_HI_2YR.isnull().sum()

224849

In [19]:
ipums_earnings.head()

,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STRATA,GQ,PERNUM,PERWT,...,GRADEATT,GRADEATTD,SCHLTYPE,DEGFIELD,DEGFIELDD,DEGFIELD2,DEGFIELD2D,INCTOT,INCWAGE,INCEARN
0,2015,201501,6,518,175.0,2015000000061,10001,1,4,201.0,...,0,0,1,0,0,0,0,4400,4400,4400
1,2015,201501,9,717,119.0,2015000000091,190001,1,1,119.0,...,0,0,1,62,6203,0,0,89500,89000,89000
2,2015,201501,11,810,12.0,2015000000111,250001,1,3,27.0,...,0,0,1,52,5200,0,0,24900,24900,24900
3,2015,201501,13,1014,111.0,2015000000131,130401,1,1,112.0,...,0,0,1,23,2304,0,0,32200,32200,32200
4,2015,201501,13,1014,111.0,2015000000131,130401,1,2,111.0,...,0,0,1,62,6212,64,6402,4500,4500,4500


In [20]:
df_earnings = ipums_earnings[['YEAR','SEX','AGE','RACE','HISPAN','SPEAKENG','LANGUAGE','EDUC','EDUCD','GRADEATTD','SCHLTYPE','DEGFIELD','DEGFIELDD','INCTOT','INCWAGE','INCEARN']]

In [21]:
df_earnings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3480188 entries, 0 to 3480187
Data columns (total 16 columns):
 #   Column     Dtype
---  ------     -----
 0   YEAR       int64
 1   SEX        int64
 2   AGE        int64
 3   RACE       int64
 4   HISPAN     int64
 5   SPEAKENG   int64
 6   LANGUAGE   int64
 7   EDUC       int64
 8   EDUCD      int64
 9   GRADEATTD  int64
 10  SCHLTYPE   int64
 11  DEGFIELD   int64
 12  DEGFIELDD  int64
 13  INCTOT     int64
 14  INCWAGE    int64
 15  INCEARN    int64
dtypes: int64(16)
memory usage: 424.8 MB


In [22]:
df_earnings.YEAR.value_counts()

2018    704564
2017    699600
2019    697780
2015    689296
2016    688948
Name: YEAR, dtype: int64

In [23]:
df_earnings.EDUCD.value_counts()

63     776292
71     711159
101    684978
65     291209
81     265992
114    201964
64     124797
50     120280
61      68561
40      50917
115     43325
30      35978
2       27574
116     26433
26      21985
23      13485
25       5259
22       2993
16       2155
17       1791
15       1145
14        724
11        661
12        531
Name: EDUCD, dtype: int64

In [24]:
df_bachelors_earnings = df_earnings.loc[df_earnings['EDUCD'] == 101]

In [25]:
df_bachelors_earnings.head()

,YEAR,SEX,AGE,RACE,HISPAN,SPEAKENG,LANGUAGE,EDUC,EDUCD,GRADEATTD,SCHLTYPE,DEGFIELD,DEGFIELDD,INCTOT,INCWAGE,INCEARN
2,2015,2,25,1,0,3,1,10,101,0,1,52,5200,24900,24900,24900
7,2015,2,27,2,0,3,1,10,101,0,1,19,1901,42000,42000,42000
23,2015,2,22,1,0,3,1,10,101,70,2,36,3600,2000,2000,2000
28,2015,1,32,1,0,3,1,10,101,0,1,62,6201,61000,67000,65500
37,2015,1,27,1,0,3,1,10,101,0,1,21,2105,40000,0,40000


In [26]:
df_bachelors_earnings = df_bachelors_earnings.drop(columns=['GRADEATTD','EDUC','EDUCD'])

In [27]:
df_bachelors_earnings.head()

,YEAR,SEX,AGE,RACE,HISPAN,SPEAKENG,LANGUAGE,SCHLTYPE,DEGFIELD,DEGFIELDD,INCTOT,INCWAGE,INCEARN
2,2015,2,25,1,0,3,1,1,52,5200,24900,24900,24900
7,2015,2,27,2,0,3,1,1,19,1901,42000,42000,42000
23,2015,2,22,1,0,3,1,2,36,3600,2000,2000,2000
28,2015,1,32,1,0,3,1,1,62,6201,61000,67000,65500
37,2015,1,27,1,0,3,1,1,21,2105,40000,0,40000


In [37]:
df_bachelors_earnings.shape

(684978, 13)

In [36]:
df_bachelors_earnings.DEGFIELD.value_counts()

62    138964
61     52006
24     49737
55     48884
23     46639
60     41633
19     41139
52     35855
36     35634
21     27939
53     19290
33     18479
50     15067
41     14155
64     11817
11      8686
34      8304
40      7644
37      7468
54      7410
13      6558
29      6532
26      6016
25      4872
14      4210
48      4078
49      3469
15      2740
20      2317
56      2279
59      2082
32      1237
22      1158
57       295
51       281
35        75
38        29
Name: DEGFIELD, dtype: int64

In [35]:
df_bachelors_earnings['INCWAGE'].value_counts()

0         78870
40000     25093
50000     24797
30000     22299
60000     20302
          ...  
343000        1
321000        1
298000        1
312000        1
301000        1
Name: INCWAGE, Length: 1012, dtype: int64

In [31]:
np.where((df_bachelors_earnings['SPEAKENG']==1 | df_bachelors_earnings['SPEAKENG']==6 | df_bachelors_earnings['SPEAKENG']==7 | df_bachelors_earnings['SPEAKENG']==8 | df_bachelors_earnings['SPEAKENG']==9), 0, 1)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().